## LangGraph Open Deep Research - Supervisor-Researcher Architecture

In this notebook, we'll explore the **supervisor-researcher delegation architecture** for conducting deep research with LangGraph.

You can visit this repository to see the original application: [Open Deep Research](https://github.com/langchain-ai/open_deep_research)

Let's jump in!

## What We're Building

This implementation uses a **hierarchical delegation pattern** where:

1. **User Clarification** - Optionally asks clarifying questions to understand the research scope
2. **Research Brief Generation** - Transforms user messages into a structured research brief
3. **Supervisor** - A lead researcher that analyzes the brief and delegates research tasks
4. **Parallel Researchers** - Multiple sub-agents that conduct focused research simultaneously
5. **Research Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings are combined into a comprehensive report

![Architecture Diagram](https://i.imgur.com/Q8HEZn0.png)

This differs from a section-based approach by allowing dynamic task decomposition based on the research question, rather than predefined sections.

---

# 🤝 Breakout Room #1
## Deep Research Foundations

In this breakout room, we'll understand the architecture and components of the Open Deep Research system.

## Task 1: Dependencies

You'll need API keys for Anthropic (for the LLM) and Tavily (for web search). We'll configure the system to use Anthropic's Claude Sonnet 4 exclusively.

In [3]:
import os
import getpass

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

## Task 2: State Definitions

The state structure is hierarchical with three levels:

### Agent State (Top Level)
Contains the overall conversation messages, research brief, accumulated notes, and final report.

### Supervisor State (Middle Level)
Manages the research supervisor's messages, research iterations, and coordinating parallel researchers.

### Researcher State (Bottom Level)
Each individual researcher has their own message history, tool call iterations, and research findings.

We also have structured outputs for tool calling:
- **ConductResearch** - Tool for supervisor to delegate research to a sub-agent
- **ResearchComplete** - Tool to signal research phase is done
- **ClarifyWithUser** - Structured output for asking clarifying questions
- **ResearchQuestion** - Structured output for the research brief

Let's import these from our library: [`open_deep_library/state.py`](open_deep_library/state.py)

In [4]:
# Import state definitions from the library
from open_deep_library.state import (
    # Main workflow states
    AgentState,           # Lines 65-72: Top-level agent state with messages, research_brief, notes, final_report
    AgentInputState,      # Lines 62-63: Input state is just messages
    
    # Supervisor states
    SupervisorState,      # Lines 74-81: Supervisor manages research delegation and iterations
    
    # Researcher states
    ResearcherState,      # Lines 83-90: Individual researcher with messages and tool iterations
    ResearcherOutputState, # Lines 92-96: Output from researcher (compressed research + raw notes)
    
    # Structured outputs for tool calling
    ConductResearch,      # Lines 15-19: Tool for delegating research to sub-agents
    ResearchComplete,     # Lines 21-22: Tool to signal research completion
    ClarifyWithUser,      # Lines 30-41: Structured output for user clarification
    ResearchQuestion,     # Lines 43-48: Structured output for research brief
)

## Task 3: Utility Functions and Tools

The system uses several key utilities:

### Search Tools
- **tavily_search** - Async web search with automatic summarization to stay within token limits
- Supports Anthropic native web search and Tavily API

### Reflection Tools
- **think_tool** - Allows researchers to reflect on their progress and plan next steps (ReAct pattern)

### Helper Utilities
- **get_all_tools** - Assembles the complete toolkit (search + MCP + reflection)
- **get_today_str** - Provides current date context for research
- Token limit handling utilities for graceful degradation

These are defined in [`open_deep_library/utils.py`](open_deep_library/utils.py)

In [5]:
# Import utility functions and tools from the library
from open_deep_library.utils import (
    # Search tool - Lines 43-136: Tavily search with automatic summarization
    tavily_search,
    
    # Reflection tool - Lines 219-244: Strategic thinking tool for ReAct pattern
    think_tool,
    
    # Tool assembly - Lines 569-597: Get all configured tools
    get_all_tools,
    
    # Date utility - Lines 872-879: Get formatted current date
    get_today_str,
    
    # Supporting utilities for error handling
    get_api_key_for_model,          # Lines 892-914: Get API keys from config or env
    is_token_limit_exceeded,         # Lines 665-701: Detect token limit errors
    get_model_token_limit,           # Lines 831-846: Look up model's token limit
    remove_up_to_last_ai_message,    # Lines 848-866: Truncate messages for retry
    anthropic_websearch_called,      # Lines 607-637: Detect Anthropic native search usage
    openai_websearch_called,         # Lines 639-658: Detect OpenAI native search usage
    get_notes_from_tool_calls,       # Lines 599-601: Extract notes from tool messages
)

## Task 4: Configuration System

The configuration system controls:

### Research Behavior
- **allow_clarification** - Whether to ask clarifying questions before research
- **max_concurrent_research_units** - How many parallel researchers can run (default: 5)
- **max_researcher_iterations** - How many times supervisor can delegate research (default: 6)
- **max_react_tool_calls** - Tool call limit per researcher (default: 10)

### Model Configuration
- **research_model** - Model for research and supervision (we'll use Anthropic)
- **compression_model** - Model for synthesizing findings
- **final_report_model** - Model for writing the final report
- **summarization_model** - Model for summarizing web search results

### Search Configuration
- **search_api** - Which search API to use (ANTHROPIC, TAVILY, or NONE)
- **max_content_length** - Character limit before summarization

Defined in [`open_deep_library/configuration.py`](open_deep_library/configuration.py)

In [6]:
# Import configuration from the library
from open_deep_library.configuration import (
    Configuration,    # Lines 38-247: Main configuration class with all settings
    SearchAPI,        # Lines 11-17: Enum for search API options (ANTHROPIC, TAVILY, NONE)
)

## Task 5: Prompt Templates

The system uses carefully engineered prompts for each phase:

### Phase 1: Clarification
**clarify_with_user_instructions** - Analyzes if the research scope is clear or needs clarification

### Phase 2: Research Brief
**transform_messages_into_research_topic_prompt** - Converts user messages into a detailed research brief

### Phase 3: Supervisor
**lead_researcher_prompt** - System prompt for the supervisor that manages delegation strategy

### Phase 4: Researcher
**research_system_prompt** - System prompt for individual researchers conducting focused research

### Phase 5: Compression
**compress_research_system_prompt** - Prompt for synthesizing research findings without losing information

### Phase 6: Final Report
**final_report_generation_prompt** - Comprehensive prompt for writing the final report

All prompts are defined in [`open_deep_library/prompts.py`](open_deep_library/prompts.py)

In [7]:
# Import prompt templates from the library
from open_deep_library.prompts import (
    clarify_with_user_instructions,                    # Lines 3-41: Ask clarifying questions
    transform_messages_into_research_topic_prompt,     # Lines 44-77: Generate research brief
    lead_researcher_prompt,                            # Lines 79-136: Supervisor system prompt
    research_system_prompt,                            # Lines 138-183: Researcher system prompt
    compress_research_system_prompt,                   # Lines 186-222: Research compression prompt
    final_report_generation_prompt,                    # Lines 228-308: Final report generation
)

## ❓ Question #1:

Explain the interrelationships between the three states (Agent, Supervisor, Researcher). Why don't we just make a single huge state?

##### Answer:
The three states are separated because each role in the system has a different responsibility.

The Agent State holds the overall context, including the user’s messages, the research brief, and the final report. It represents the global view of the task.

The Supervisor State manages coordination. It keeps track of research iterations and decides how to delegate work to researchers and when the research is complete.

The Researcher State is used by individual sub-agents to do focused research. Each researcher has its own state so they can work independently without interfering with each other.

Using a single large state would mix all responsibilities together, increase token usage, and make parallel research unreliable. Separating the states keeps the system organized, efficient, and easier to reason about.


## ❓ Question #2:

What are the advantages and disadvantages of importing these components instead of including them in the notebook?

##### Answer:
Importing components keeps the notebook clean and focused on the workflow instead of implementation details. The logic is easier to reuse, test, and maintain when it lives in separate Python files rather than inside a notebook.

It also makes the system more production-ready, since the same components can be used outside the notebook in an application or service.

The downside is that the code can feel less transparent for beginners, because important logic is hidden in external files. It also requires jumping between files to fully understand what is happening.


## 🏗️ Activity #1: Explore the Prompts

Open `open_deep_library/prompts.py` and examine one of the prompt templates in detail.

**Requirements:**
1. Choose one prompt template (clarify, brief, supervisor, researcher, compression, or final report)
2. Explain what the prompt is designed to accomplish
3. Identify 2-3 key techniques used in the prompt (e.g., structured output, role definition, examples)
4. Suggest one improvement you might make to the prompt

**YOUR CODE HERE** - Write your analysis in a markdown cell below

## 🏗️ Chosen prompt template: lead_researcher_prompt 
#### 1) What the prompt is designed to accomplish:

This prompt is designed to turn the language model into a research supervisor, not a researcher. Its main goal is to plan and coordinate the research process, rather than collecting information itself.

Instead of answering the question directly, the supervisor decides:


    - how the research should be broken down,
    - whether the task can be handled by one researcher or needs multiple parallel researchers,
    - when enough information has been gathered to stop researching

The prompt enforces a clear workflow where the supervisor delegates research using tools, evaluates the results, and then signals completion when the findings are sufficient.

#### 2) Key techniques used in the prompt: 

a) Strong role definition

The prompt clearly defines the model’s role as a research supervisor. It explicitly states that the supervisor’s job is to delegate research and manage progress, not to write the final report. This prevents the model from drifting into free-form answering.

b) Structured tool usage

The research supervisor is required to use specific tools (ConductResearch, ResearchComplete, and think_tool). This ensures that actions like delegation and completion happen in a controlled, predictable way rather than through unstructured text.

c) Reflection and limits

The prompt forces the supervisor to pause and reflect using think_tool before and after delegating tasks. It also introduces hard limits on the number of delegations and parallel agents, which helps control cost, token usage, and infinite research loops.

#### 3) One improvement suggestion:

One improvement would be to define clearer and more consistent output requirements for each delegated research task. For example, the supervisor could explicitly instruct each researcher to return a fixed number of key findings (e.g. 3–5 points) along with a specific number of sources for each finding.

This would help standardize the quality and structure of the research outputs. Currently, different researchers may return results in varying levels of detail or formats, which makes it harder to merge their findings later. By setting clear expectations upfront, the supervisor would receive more predictable and comparable outputs.

As a result, the compression and final report stages would become simpler and more reliable, because the system would be combining well-structured research summaries rather than inconsistent free-form notes.

---

# 🤝 Breakout Room #2
## Building & Running the Researcher

In this breakout room, we'll explore the node functions, build the graph, and run wellness research.

## Task 6: Node Functions - The Building Blocks

Now let's look at the node functions that make up our graph. We'll import them from the library and understand what each does.

### The Complete Research Workflow

The workflow consists of 8 key nodes organized into 3 subgraphs:

1. **Main Graph Nodes:**
   - `clarify_with_user` - Entry point that checks if clarification is needed
   - `write_research_brief` - Transforms user input into structured research brief
   - `final_report_generation` - Synthesizes all research into final report

2. **Supervisor Subgraph Nodes:**
   - `supervisor` - Lead researcher that plans and delegates
   - `supervisor_tools` - Executes supervisor's tool calls (delegation, reflection)

3. **Researcher Subgraph Nodes:**
   - `researcher` - Individual researcher conducting focused research
   - `researcher_tools` - Executes researcher's tool calls (search, reflection)
   - `compress_research` - Synthesizes researcher's findings

All nodes are defined in [`open_deep_library/deep_researcher.py`](open_deep_library/deep_researcher.py)

### Node 1: clarify_with_user

**Purpose:** Analyzes user messages and asks clarifying questions if the research scope is unclear.

**Key Steps:**
1. Check if clarification is enabled in configuration
2. Use structured output to analyze if clarification is needed
3. If needed, end with a clarifying question for the user
4. If not needed, proceed to research brief with verification message

**Implementation:** [`open_deep_library/deep_researcher.py` lines 60-115](open_deep_library/deep_researcher.py#L60-L115)

In [8]:
# Import the clarify_with_user node
from open_deep_library.deep_researcher import clarify_with_user

### Node 2: write_research_brief

**Purpose:** Transforms user messages into a structured research brief for the supervisor.

**Key Steps:**
1. Use structured output to generate detailed research brief from messages
2. Initialize supervisor with system prompt and research brief
3. Set up supervisor messages with proper context

**Why this matters:** A well-structured research brief helps the supervisor make better delegation decisions.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 118-175](open_deep_library/deep_researcher.py#L118-L175)

In [10]:
# Import the write_research_brief node
from open_deep_library.deep_researcher import write_research_brief

### Node 3: supervisor

**Purpose:** Lead research supervisor that plans research strategy and delegates to sub-researchers.

**Key Steps:**
1. Configure model with three tools:
   - `ConductResearch` - Delegate research to a sub-agent
   - `ResearchComplete` - Signal that research is done
   - `think_tool` - Strategic reflection before decisions
2. Generate response based on current context
3. Increment research iteration count
4. Proceed to tool execution

**Decision Making:** The supervisor uses `think_tool` to reflect before delegating research, ensuring thoughtful decomposition of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 178-223](open_deep_library/deep_researcher.py#L178-L223)

In [11]:
# Import the supervisor node (from supervisor subgraph)
from open_deep_library.deep_researcher import supervisor

### Node 4: supervisor_tools

**Purpose:** Executes the supervisor's tool calls, including strategic thinking and research delegation.

**Key Steps:**
1. Check exit conditions:
   - Exceeded maximum iterations
   - No tool calls made
   - `ResearchComplete` called
2. Process `think_tool` calls for strategic reflection
3. Execute `ConductResearch` calls in parallel:
   - Spawn researcher subgraphs for each delegation
   - Limit to `max_concurrent_research_units` (default: 5)
   - Gather all results asynchronously
4. Aggregate findings and return to supervisor

**Parallel Execution:** This is where the magic happens - multiple researchers work simultaneously on different aspects of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 225-349](open_deep_library/deep_researcher.py#L225-L349)

In [12]:
# Import the supervisor_tools node
from open_deep_library.deep_researcher import supervisor_tools

### Node 5: researcher

**Purpose:** Individual researcher that conducts focused research on a specific topic.

**Key Steps:**
1. Load all available tools (search, MCP, reflection)
2. Configure model with tools and researcher system prompt
3. Generate response with tool calls
4. Increment tool call iteration count

**ReAct Pattern:** Researchers use `think_tool` to reflect after each search, deciding whether to continue or provide their answer.

**Available Tools:**
- Search tools (Tavily or Anthropic native search)
- `think_tool` for strategic reflection
- `ResearchComplete` to signal completion
- MCP tools (if configured)

**Implementation:** [`open_deep_library/deep_researcher.py` lines 365-424](open_deep_library/deep_researcher.py#L365-L424)

In [13]:
# Import the researcher node (from researcher subgraph)
from open_deep_library.deep_researcher import researcher

### Node 6: researcher_tools

**Purpose:** Executes the researcher's tool calls, including searches and strategic reflection.

**Key Steps:**
1. Check early exit conditions (no tool calls, native search used)
2. Execute all tool calls in parallel:
   - Search tools fetch and summarize web content
   - `think_tool` records strategic reflections
   - MCP tools execute external integrations
3. Check late exit conditions:
   - Exceeded `max_react_tool_calls` (default: 10)
   - `ResearchComplete` called
4. Continue research loop or proceed to compression

**Error Handling:** Safely handles tool execution errors and continues with available results.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 435-509](open_deep_library/deep_researcher.py#L435-L509)

In [14]:
# Import the researcher_tools node
from open_deep_library.deep_researcher import researcher_tools

### Node 7: compress_research

**Purpose:** Compresses and synthesizes research findings into a concise, structured summary.

**Key Steps:**
1. Configure compression model
2. Add compression instruction to messages
3. Attempt compression with retry logic:
   - If token limit exceeded, remove older messages
   - Retry up to 3 times
4. Extract raw notes from tool and AI messages
5. Return compressed research and raw notes

**Why Compression?** Researchers may accumulate lots of tool outputs and reflections. Compression ensures:
- All important information is preserved
- Redundant information is deduplicated
- Content stays within token limits for the final report

**Token Limit Handling:** Gracefully handles token limit errors by progressively truncating messages.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 511-585](open_deep_library/deep_researcher.py#L511-L585)

In [15]:
# Import the compress_research node
from open_deep_library.deep_researcher import compress_research

### Node 8: final_report_generation

**Purpose:** Generates the final comprehensive research report from all collected findings.

**Key Steps:**
1. Extract all notes from completed research
2. Configure final report model
3. Attempt report generation with retry logic:
   - If token limit exceeded, truncate findings by 10%
   - Retry up to 3 times
4. Return final report or error message

**Token Limit Strategy:**
- First retry: Use model's token limit × 4 as character limit
- Subsequent retries: Reduce by 10% each time
- Graceful degradation with helpful error messages

**Report Quality:** The prompt guides the model to create well-structured reports with:
- Proper headings and sections
- Inline citations
- Comprehensive coverage of all findings
- Sources section at the end

**Implementation:** [`open_deep_library/deep_researcher.py` lines 607-697](open_deep_library/deep_researcher.py#L607-L697)

In [16]:
# Import the final_report_generation node
from open_deep_library.deep_researcher import final_report_generation

## Task 7: Graph Construction - Putting It All Together

The system is organized into three interconnected graphs:

### 1. Researcher Subgraph (Bottom Level)
Handles individual focused research on a specific topic:
```
START → researcher → researcher_tools → compress_research → END
               ↑            ↓
               └────────────┘ (loops until max iterations or ResearchComplete)
```

### 2. Supervisor Subgraph (Middle Level)
Manages research delegation and coordination:
```
START → supervisor → supervisor_tools → END
            ↑              ↓
            └──────────────┘ (loops until max iterations or ResearchComplete)
            
supervisor_tools spawns multiple researcher_subgraphs in parallel
```

### 3. Main Deep Researcher Graph (Top Level)
Orchestrates the complete research workflow:
```
START → clarify_with_user → write_research_brief → research_supervisor → final_report_generation → END
                 ↓                                       (supervisor_subgraph)
               (may end early if clarification needed)
```

Let's import the compiled graphs from the library.

In [17]:
# Import the pre-compiled graphs from the library
from open_deep_library.deep_researcher import (
    # Bottom level: Individual researcher workflow
    researcher_subgraph,    # Lines 588-605: researcher → researcher_tools → compress_research
    
    # Middle level: Supervisor coordination
    supervisor_subgraph,    # Lines 351-363: supervisor → supervisor_tools (spawns researchers)
    
    # Top level: Complete research workflow
    deep_researcher,        # Lines 699-719: Main graph with all phases
)

## Why This Architecture?

### Advantages of Supervisor-Researcher Delegation

1. **Dynamic Task Decomposition**
   - Unlike section-based approaches with predefined structure, the supervisor can break down research based on the actual question
   - Adapts to different types of research (comparisons, lists, deep dives, etc.)

2. **Parallel Execution**
   - Multiple researchers work simultaneously on different aspects
   - Much faster than sequential section processing
   - Configurable parallelism (1-20 concurrent researchers)

3. **ReAct Pattern for Quality**
   - Researchers use `think_tool` to reflect after each search
   - Prevents excessive searching and improves search quality
   - Natural stopping conditions based on information sufficiency

4. **Flexible Tool Integration**
   - Easy to add MCP tools for specialized research
   - Supports multiple search APIs (Anthropic, Tavily)
   - Each researcher can use different tool combinations

5. **Graceful Token Limit Handling**
   - Compression prevents token overflow
   - Progressive truncation in final report generation
   - Research can scale to arbitrary depths

### Trade-offs

- **Complexity:** More moving parts than section-based approach
- **Cost:** Parallel researchers use more tokens (but faster)
- **Unpredictability:** Research structure emerges dynamically

## Task 8: Running the Deep Researcher

Now let's see the system in action! We'll use it to research wellness strategies for improving sleep quality.

### Setup

We need to:
1. Set up the wellness research request
2. Configure the execution with Anthropic settings
3. Run the research workflow

In [18]:
# Set up the graph with Anthropic configuration
from IPython.display import Markdown, display
import uuid

# Note: deep_researcher is already compiled from the library
# For this demo, we'll use it directly without additional checkpointing
graph = deep_researcher

print("✓ Graph ready for execution")
print("  (Note: The graph is pre-compiled from the library)")

✓ Graph ready for execution
  (Note: The graph is pre-compiled from the library)


### Configuration for Anthropic

We'll configure the system to use:
- **Claude Sonnet 4** for all research, supervision, and report generation
- **Tavily** for web search (you can also use Anthropic's native search)
- **Moderate parallelism** (1 concurrent researcher for cost control)
- **Clarification enabled** (will ask if research scope is unclear)

In [19]:
# Configure for Anthropic with moderate settings
config = {
    "configurable": {
        # Model configuration - using Claude Sonnet 4 for everything
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 10000,
        
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        
        # Research behavior
        "allow_clarification": True,
        "max_concurrent_research_units": 1,  # 1 parallel researcher
        "max_researcher_iterations": 2,      # Supervisor can delegate up to 2 times
        "max_react_tool_calls": 3,           # Each researcher can make up to 3 tool calls
        
        # Search configuration
        "search_api": "tavily",  # Using Tavily for web search
        "max_content_length": 50000,
        
        # Thread ID for this conversation
        "thread_id": str(uuid.uuid4())
    }
}

print("✓ Configuration ready")
print(f"  - Research Model: Claude Sonnet 4")
print(f"  - Max Concurrent Researchers: 1")
print(f"  - Max Iterations: 2")
print(f"  - Search API: Tavily")

✓ Configuration ready
  - Research Model: Claude Sonnet 4
  - Max Concurrent Researchers: 1
  - Max Iterations: 2
  - Search API: Tavily


### Execute the Wellness Research

Now let's run the research! We'll ask the system to research evidence-based strategies for improving sleep quality.

The workflow will:
1. **Clarify** - Check if the request is clear (may skip if obvious)
2. **Research Brief** - Transform our request into a structured brief
3. **Supervisor** - Plan research strategy and delegate to researchers
4. **Parallel Research** - Researchers gather information simultaneously
5. **Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings combined into comprehensive report

In [20]:
# Create our wellness research request
research_request = """
I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please research the best evidence-based strategies for improving sleep quality and create a comprehensive sleep improvement plan for me.
"""

# Execute the graph
async def run_research():
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run the research
await run_research()

Starting research workflow...


Node: clarify_with_user

I have sufficient information to proceed with your sleep improvement research request. I understand you're looking for evidence-based strategies to address your current sleep challenges, which include inconsistent bedtimes (10pm-1am), phone usage in bed, and morning fatigue. I'll now research the best scientific approaches to improving sleep quality and create a comprehensive, personalized sleep improvement plan for you.

Node: write_research_brief

Research Brief Generated:
I want to improve my sleep quality based on evidence-based strategies. My current sleep challenges include: going to bed at inconsistent times (ranging from 10pm to 1am), using my phone in bed, and often feeling tired in the morning despite getting sleep. Please research and compile a comprehensive sleep improvement plan that addresses these specific issues. The research should focus on: 1) Scientific evidence for optimal sleep hygiene practices, particularly


Node: research_supervisor

Node: final_report_generation

FINAL REPORT GENERATED



Error generating final report: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': "This request would exceed your organization's rate limit of 30,000 input tokens per minute (org: dbcb5ef4-057e-4277-80f5-00232afb714a, model: claude-sonnet-4-20250514). For details, refer to: https://docs.claude.com/en/api/rate-limits. You can see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase."}, 'request_id': 'req_011CXtVFNFt9YtvisDPCPxyo'}


Research workflow completed!


### 🏗️ Testing a More Conservative Configuration

To reduce token limit warnings and make the execution more predictable for demonstration purposes, I’ll run the workflow with a more conservative configuration. This setup prioritizes clarity, cost control, and stability over maximum depth or parallelism.



In [23]:
# Configure for Anthropic with moderate settings
config = {
    "configurable": {
        # Model configuration - using Claude Sonnet 4 for everything
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 8000,
        
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 6000,
        
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 8000,
        
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 6000,
        
        # Research behavior
        "allow_clarification": True,
        "max_concurrent_research_units": 1,  # 1 parallel researcher
        "max_researcher_iterations": 1,      # Supervisor can delegate up to 1 times
        "max_react_tool_calls": 2,           # Each researcher can make up to 2 tool calls
        
        # Search configuration
        "search_api": "tavily",  # Using Tavily for web search
        "max_content_length": 40000,
        
        # Thread ID for this conversation
        "thread_id": str(uuid.uuid4())
    }
}

print("✓ Configuration ready")
print(f"  - Research Model: Claude Sonnet 4")
print(f"  - Max Concurrent Researchers: 1")
print(f"  - Max Iterations: 1")
print(f"  - Search API: Tavily")

✓ Configuration ready
  - Research Model: Claude Sonnet 4
  - Max Concurrent Researchers: 1
  - Max Iterations: 1
  - Search API: Tavily


### 🏗️ Running the Wellness Research with the Updated Configuration

With the updated configuration in place, we now rerun the wellness research request. The system will follow the same deep research workflow, but with tighter limits on iterations and tool calls to ensure a clean and stable execution.


In [22]:
# Create our wellness research request
research_request = """
I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please research the best evidence-based strategies for improving sleep quality and create a comprehensive sleep improvement plan for me.
"""

# Execute the graph
async def run_research():
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run the research
await run_research()

Starting research workflow...


Node: clarify_with_user

I have sufficient information to proceed with researching evidence-based sleep improvement strategies for you. I understand that you currently have inconsistent bedtimes (10pm-1am), use your phone in bed, and often feel tired in the morning. I will research the best evidence-based strategies for improving sleep quality and create a comprehensive, personalized sleep improvement plan that addresses your specific challenges. I'll now begin the research process.

Node: write_research_brief

Research Brief Generated:
I want to improve my sleep quality and need a comprehensive, evidence-based sleep improvement plan. My current sleep challenges include: going to bed at inconsistent times (ranging from 10pm to 1am), using my phone in bed, and often feeling tired in the morning despite sleep. Please research the most effective, scientifically-backed strategies for improving sleep quality that specifically address inconsistent bedtime sche

# Evidence-Based Sleep Improvement Plan: Addressing Inconsistent Bedtimes, Technology Use, and Morning Fatigue

## Understanding Your Sleep Challenges

Your current sleep difficulties—inconsistent bedtimes, phone use in bed, and morning fatigue—are interconnected issues that significantly impact sleep quality. Research from the American Academy of Sleep Medicine demonstrates that irregular sleep-wake patterns disrupt circadian rhythms, while blue light exposure from electronic devices suppresses melatonin production by up to 23% [1]. Morning fatigue despite adequate sleep duration often indicates poor sleep quality rather than insufficient sleep quantity, frequently resulting from fragmented sleep cycles caused by these behavioral patterns.

The variability in your bedtime (10pm-1am) creates a condition similar to "social jet lag," where your internal biological clock becomes misaligned with your sleep schedule. Studies published in Sleep Medicine Reviews show that bedtime variability of more than 30 minutes can reduce sleep efficiency and increase daytime sleepiness [2]. Your phone use in bed compounds this issue by providing both psychological arousal through content consumption and physiological disruption through blue light exposure during the critical pre-sleep period.

## Optimal Sleep Timing and Consistency Practices

### Establishing Your Ideal Sleep Schedule

Research from the Journal of Clinical Medicine indicates that consistency is more important than the specific bedtime chosen, provided it aligns with your natural chronotype [3]. For most adults, a bedtime between 10:00-11:00 PM optimizes alignment with natural melatonin production, which typically begins around 9:00 PM. Given your current range of 10pm-1am, targeting a consistent 10:30 PM bedtime represents a realistic compromise that doesn't require dramatic schedule shifts.

The National Sleep Foundation recommends implementing a "15-minute rule" for schedule adjustment: shift your bedtime by only 15 minutes earlier each night until you reach your target time [4]. This gradual approach prevents the circadian disruption that occurs with sudden schedule changes. For your situation, if you're currently going to bed at 1:00 AM, plan a 2-3 week transition period to reach 10:30 PM consistently.

### Weekend Sleep Schedule Management

Clinical sleep research published in Sleep Health demonstrates that weekend sleep schedule variations should not exceed 1 hour from weekday schedules to maintain circadian stability [5]. This means if your target bedtime is 10:30 PM on weeknights, weekend bedtimes should remain between 10:00-11:30 PM. Similarly, wake times should vary by no more than 1 hour on weekends to prevent "Sunday night insomnia" and Monday morning fatigue.

### Implementation Strategy for Consistency

Sleep medicine experts recommend using "anchor points" to establish consistency: fixed wake times are more powerful than fixed bedtimes for regulating circadian rhythms [6]. Choose a wake time you can maintain 7 days per week, then work backward 7-9 hours to determine your ideal bedtime. Use bright light exposure within 30 minutes of waking and dim lighting 2 hours before bedtime to reinforce your chosen schedule.

## Evidence-Based Methods for Reducing Phone and Screen Usage Before Bed

### Understanding the Science of Blue Light Impact

Research published in the Proceedings of the National Academy of Sciences shows that blue light exposure (particularly wavelengths between 460-480nm) suppresses melatonin production for up to 3 hours after exposure [7]. Smartphones emit significant blue light in this range, with studies indicating that 2 hours of evening tablet use can delay melatonin onset by approximately 1.5 hours. This delay directly correlates with increased sleep onset latency and reduced REM sleep quality.

### Practical Digital Sunset Strategies

The American Academy of Sleep Medicine recommends implementing a "digital sunset" 1-2 hours before bedtime [8]. For your 10:30 PM target bedtime, this means ceasing phone use by 8:30-9:30 PM. Research from Harvard Medical School suggests a graduated approach works better than complete elimination:

**Week 1-2**: Move your phone charger outside the bedroom and use an analog alarm clock. This removes the temptation for bedtime phone use while maintaining functionality.

**Week 3-4**: Implement a "phone parking" station in your living room where all devices charge overnight, creating a physical barrier to evening use.

**Week 5+**: Establish phone-free evening activities during your digital sunset period, such as reading, gentle stretching, or preparation for the next day.

### Technology-Based Solutions

If complete elimination isn't immediately feasible, research supports harm-reduction approaches. Studies in Applied Ergonomics show that blue light filtering glasses can reduce melatonin suppression by up to 58% when worn 2-3 hours before bedtime [9]. Additionally, enabling "night mode" or blue light filters on devices after sunset can reduce circadian disruption, though complete avoidance remains optimal.

## Strategies to Improve Morning Energy Levels

### Addressing Sleep Inertia

Morning fatigue despite adequate sleep duration often indicates "sleep inertia," a neurophysiological state lasting 15 minutes to 4 hours after awakening. Research in Sleep Medicine Clinics identifies several evidence-based interventions for reducing sleep inertia [10]:

**Strategic Light Exposure**: Bright light therapy (2500-10000 lux) within 30 minutes of waking can reduce morning grogginess by up to 50%. Position yourself near a window or use a light therapy device for 20-30 minutes while having breakfast or coffee.

**Consistent Wake Times**: Studies show that waking at the same time daily, including weekends, reduces sleep inertia severity by maintaining circadian alignment. Set your alarm for the same time every day, regardless of bedtime variations during your transition period.

### Optimizing Sleep Architecture

Morning energy levels depend heavily on sleep stage timing at awakening. Research from the Journal of Sleep Research indicates that waking during light sleep stages (N1/N2) rather than deep sleep (N3) or REM sleep significantly improves morning alertness [11]. Since natural sleep cycles average 90-110 minutes, timing your sleep for complete cycles can improve morning energy:

- For a 6:30 AM wake time, optimal bedtimes are 9:00 PM, 10:30 PM, or 12:00 AM (allowing for 15-20 minutes to fall asleep)
- Track your natural awakening patterns during weekends to identify your personal sleep cycle length

### Morning Routine Optimization

Clinical research supports specific morning practices for enhancing alertness and energy. The Sleep Research Society recommends a morning routine including [12]:

**Immediate light exposure**: Open curtains or go outside within 15 minutes of waking to suppress residual melatonin and activate cortisol production.

**Hydration**: Drink 16-20 oz of water upon waking to address overnight dehydration, which contributes to morning fatigue.

**Gentle movement**: 5-10 minutes of light stretching or walking increases circulation and accelerates the transition from sleep to wake physiology.

**Delayed caffeine**: Wait 60-90 minutes after waking before consuming caffeine to allow natural cortisol awakening response and prevent afternoon crashes.

## Comprehensive Sleep Hygiene Techniques

### Sleep Environment Optimization

The National Sleep Foundation's sleep environment research identifies several critical factors for optimal sleep quality [13]:

**Temperature Control**: Maintain bedroom temperature between 65-68°F (18-20°C). Core body temperature naturally drops during sleep onset, and a cool environment facilitates this process. Studies show that temperatures above 70°F or below 60°F can significantly fragment sleep.

**Darkness Optimization**: Complete darkness is crucial for maintaining natural melatonin production. Use blackout curtains or eye masks to eliminate light pollution. Even small amounts of light (such as from electronic device indicators) can disrupt sleep architecture.

**Noise Management**: Maintain bedroom noise levels below 35 decibels when possible. If environmental noise is unavoidable, consistent white noise or earplugs can mask disruptive sounds without creating sleep fragmentation.

### Pre-Sleep Routine Development

Research in Behavioral Sleep Medicine demonstrates that consistent pre-sleep routines can reduce sleep onset time by 37% within 2-4 weeks of implementation [14]. Your routine should begin 60-90 minutes before bedtime and include only relaxing, non-stimulating activities:

**60-90 minutes before bed**: Begin dimming lights throughout your home and start your digital sunset period.

**45-60 minutes before bed**: Engage in calming activities such as reading (physical books only), gentle stretching, or preparation for the next day.

**15-30 minutes before bed**: Practice relaxation techniques such as progressive muscle relaxation, deep breathing exercises, or meditation.

### Dietary and Exercise Considerations

Sleep research identifies several dietary and activity factors that significantly impact sleep quality:

**Meal Timing**: Finish eating large meals at least 3 hours before bedtime. Late eating can disrupt circadian rhythms and cause sleep fragmentation. If evening hunger occurs, limit intake to small, easily digestible snacks.

**Exercise Timing**: Regular moderate exercise improves sleep quality by 65% in most individuals, but timing matters [15]. Morning or afternoon exercise enhances nighttime sleep, while vigorous exercise within 3 hours of bedtime can increase sleep onset time and reduce sleep efficiency.

**Caffeine Management**: Limit caffeine intake after 2:00 PM, as caffeine has a half-life of 5-7 hours and can disrupt sleep even when consumed 6 hours before bedtime.

### Sleep Position and Mattress Considerations

Orthopedic sleep research indicates that sleep position affects both sleep quality and morning energy levels. Side sleeping (particularly left side) promotes better circulation and can reduce sleep apnea symptoms that contribute to morning fatigue [16]. Ensure your pillow maintains neutral spine alignment and replace mattresses every 7-10 years to maintain proper support.

## Implementation Timeline and Monitoring

### Phase 1 (Weeks 1-2): Foundation Building
- Establish consistent wake time (7 days/week)
- Move phone charging station outside bedroom
- Begin 15-minute bedtime adjustments toward 10:30 PM target
- Implement basic sleep environment optimizations (temperature, darkness)

### Phase 2 (Weeks 3-4): Routine Development
- Establish complete digital sunset routine
- Develop consistent 60-minute pre-sleep routine
- Fine-tune bedroom environment (noise, comfort)
- Monitor morning energy levels and adjust as needed

### Phase 3 (Weeks 5-8): Optimization and Maintenance
- Achieve consistent 10:30 PM bedtime
- Refine morning routine for optimal energy
- Address any remaining sleep quality issues
- Establish long-term maintenance strategies

### Progress Monitoring

Track your sleep improvement using both subjective and objective measures. Keep a sleep diary noting bedtime, sleep onset time, nighttime awakenings, wake time, and morning energy levels (rated 1-10). Research shows that individuals who track their sleep improvement are 40% more likely to maintain positive changes long-term [17].

## Sources

[1] Light Suppression of Melatonin: American Academy of Sleep Medicine Clinical Practice Guidelines: https://aasm.org/clinical-resources/practice-standards/practice-guidelines/

[2] Sleep Regularity and Sleep-Wake Timing: Sleep Medicine Reviews: https://www.journals.elsevier.com/sleep-medicine-reviews

[3] Individual Differences in Sleep Timing and Chronotype: Journal of Clinical Medicine: https://www.mdpi.com/journal/jcm

[4] Sleep Schedule Recommendations: National Sleep Foundation: https://www.sleepfoundation.org/

[5] Weekend Sleep Patterns and Circadian Health: Sleep Health Journal: https://www.sleephealthjournal.org/

[6] Circadian Rhythm Regulation Through Light and Dark: Sleep Medicine Clinics: https://www.sleep.theclinics.com/

[7] Blue Light Effects on Melatonin: Proceedings of the National Academy of Sciences: https://www.pnas.org/

[8] Technology and Sleep Guidelines: American Academy of Sleep Medicine: https://aasm.org/

[9] Blue Light Filtering and Melatonin Production: Applied Ergonomics: https://www.journals.elsevier.com/applied-ergonomics

[10] Sleep Inertia Research: Sleep Medicine Clinics: https://www.sleep.theclinics.com/

[11] Sleep Architecture and Morning Alertness: Journal of Sleep Research: https://onlinelibrary.wiley.com/journal/13652869

[12] Morning Routine Recommendations: Sleep Research Society: https://www.sleepresearchsociety.org/

[13] Sleep Environment Guidelines: National Sleep Foundation: https://www.sleepfoundation.org/

[14] Pre-Sleep Routine Research: Behavioral Sleep Medicine: https://www.tandfonline.com/toc/hbsm20/current

[15] Exercise and Sleep Quality Studies: American College of Sports Medicine: https://www.acsm.org/

[16] Sleep Position and Health Research: Journal of Clinical Sleep Medicine: https://jcsm.aasm.org/

[17] Sleep Tracking and Behavior Change: Sleep Health Research: https://www.sleephealthjournal.org/


Research workflow completed!


## Task 9: Understanding the Output

Let's break down what happened:

### Phase 1: Clarification
The system checked if your request was clear. Since you provided specific details about your sleep issues, it likely proceeded without asking clarifying questions.

### Phase 2: Research Brief
Your request was transformed into a detailed research brief that guides the supervisor's delegation strategy.

### Phase 3: Supervisor Delegation
The supervisor analyzed the brief and decided how to break down the research:
- Used `think_tool` to plan strategy
- Called `ConductResearch` to delegate to researchers
- Each delegation specified a focused research topic (e.g., sleep hygiene, circadian rhythm, blue light effects)

### Phase 4: Parallel Research
Researchers worked on their assigned topics:
- Each researcher used web search tools to gather information
- Used `think_tool` to reflect after each search
- Decided when they had enough information
- Compressed their findings into clean summaries

### Phase 5: Final Report
All research findings were synthesized into a comprehensive sleep improvement plan with:
- Well-structured sections
- Evidence-based recommendations
- Practical action items
- Sources for further reading

## Task 10: Key Takeaways & Next Steps

### Architecture Benefits
1. **Dynamic Decomposition** - Research structure emerges from the question, not predefined
2. **Parallel Efficiency** - Multiple researchers work simultaneously
3. **ReAct Quality** - Strategic reflection improves search decisions
4. **Scalability** - Handles token limits gracefully through compression
5. **Flexibility** - Easy to add new tools and capabilities

### When to Use This Pattern
- **Complex research questions** that need multi-angle investigation
- **Comparison tasks** where parallel research on different topics is beneficial
- **Open-ended exploration** where structure should emerge dynamically
- **Time-sensitive research** where parallel execution speeds up results

### When to Use Section-Based Instead
- **Highly structured reports** with predefined format requirements
- **Template-based content** where sections are always the same
- **Sequential dependencies** where later sections depend on earlier ones
- **Budget constraints** where token efficiency is critical

### Extend the System
1. **Add MCP Tools** - Integrate specialized tools for your domain
2. **Custom Prompts** - Modify prompts for specific research types
3. **Different Models** - Try different Claude versions or mix models
4. **Persistence** - Use a real database for checkpointing instead of memory

### Learn More
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Open Deep Research Repo](https://github.com/langchain-ai/open_deep_research)
- [Anthropic Claude Documentation](https://docs.anthropic.com/)
- [Tavily Search API](https://tavily.com/)

## ❓ Question #3:

What are the trade-offs of using parallel researchers vs. sequential research? When might you choose one approach over the other?

##### Answer:
Using parallel researchers allows the system to explore multiple aspects of a question at the same time, which significantly reduces overall execution time and is well suited for broad or open-ended research. It also helps uncover diverse perspectives that might be missed in a strictly sequential approach.

The trade-off is higher cost and increased complexity, since running multiple researchers in parallel consumes more tokens and can be harder to reason about or debug. Parallel execution can also introduce some unpredictability in how the research is structured.

Sequential research is more cost-efficient and easier to control, making it a better choice for narrowly defined questions, highly structured tasks, or situations with strict budget constraints.


## ❓ Question #4:

How would you adapt this deep research architecture for a production wellness application? What additional components would you need?

##### Answer:
To adapt this deep research architecture for a production wellness app, I would keep the same supervisor–researcher workflow, but add product-grade layers around safety, reliability, personalization, and persistence.

Key additions:

User profile + consent layer: Store user goals, constraints, and preferences (with explicit consent) and use them to generate a better research brief and recommendations.

Safety + medical guardrails: A policy layer that detects medical/high-risk content, adds appropriate disclaimers, encourages professional help when needed, and prevents unsafe advice. For a wellness domain, this is critical.

Persistent storage + audit trail: Save briefs, tool results, citations, and final reports in a database so users can revisit past plans and the system can support debugging/compliance.

Observability + evals: Logging, tracing, and automated evaluations (quality, citation coverage, hallucination checks) plus monitoring for latency/cost regressions.

Source quality controls: Domain allowlists/blocklists, ranking heuristics, and citation verification to prefer reputable medical sources and reduce misinformation.

Caching + rate limiting: Cache search and summarization results and add rate limits per user to control cost and avoid abuse.

Human-in-the-loop workflows: Escalation paths for sensitive cases (e.g., sleep issues + severe symptoms) and optional clinician review for high-impact recommendations.

UX and delivery layer: A frontend that turns outputs into actionable plans (checklists, reminders, habit tracking) and supports follow-up questions in the same thread.

In short, the core graph stays, but production requires guardrails, persistence, monitoring, source governance, and user-facing workflows to make the research outputs safe, repeatable, and useful.


## 🏗️ Activity #2: Custom Wellness Research

Using what you've learned, run a custom wellness research task.

**Requirements:**
1. Create a wellness-related research question (exercise, nutrition, stress, etc.)
2. Modify the configuration for your use case
3. Run the research and analyze the output
4. Document what worked well and what could be improved

**Experiment ideas:**
- Research exercise routines for specific conditions (bad knee, lower back pain)
- Compare different stress management techniques
- Investigate nutrition strategies for specific goals
- Explore meditation and mindfulness research

**YOUR CODE HERE**

### 🏗️ Custom Wellness Research Task

In this experiment, I ran a custom deep research query focused on creating a practical, evidence-based plan with clear action steps. I kept conservative execution limits (low tool calls and limited iterations) to control cost and reduce token-limit issues, while still aiming for useful recommendations and reputable sources.


#### Step 1: Custom Configuration for the Wellness Research Experiment



In [27]:
# Configuration for custom wellness research (conservative demo setup)
import uuid

custom_config = {
    "configurable": {
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 8000,
        
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 6000,
        
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 8000,
        
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 6000,
        
        "allow_clarification": True,
        "max_concurrent_research_units": 1,
        "max_researcher_iterations": 1,
        "max_react_tool_calls": 2,
        
        "search_api": "tavily",
        "max_content_length": 40000,
        
        "thread_id": str(uuid.uuid4())
    }
}

print("✓ Configuration ready")
print(f"  - Research Model: Claude Sonnet 4")
print(f"  - Max Concurrent Researchers: 1")
print(f"  - Max Iterations: 1")
print(f"  - Search API: Tavily")

✓ Configuration ready
  - Research Model: Claude Sonnet 4
  - Max Concurrent Researchers: 1
  - Max Iterations: 1
  - Search API: Tavily


#### Step 2: Custom Wellness Research Request

In [28]:
from IPython.display import Markdown, display

# Create our custom wellness research request
research_request = """
Create an evidence-based 8-week plan to improve aerobic base for a beginner runner
who can currently run ~30 minutes continuously at ~6:45 min/km.

Context:
- Runs 3 times per week
- Does home workouts (bodyweight / dumbbells)
- Goal is gradual improvement without injury

Please include:
- Week-by-week running structure
- Intensity guidance (easy vs moderate vs harder efforts)
- Simple strength training recommendations
- Injury prevention tips (calves, shins, knees)
- Safety notes and when to stop training
- Sources / references
"""

# Execute the graph
async def run_research():
    """Run the custom wellness research workflow and display results."""
    print("Starting custom wellness research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        custom_config,
        stream_mode="updates"
    ):
        # Display progress for each node
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\nClarification step output:")
                    print(last_msg.content)
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print("\nResearch brief generated:")
                    print(node_output["research_brief"][:500] + "...")
            
            elif node_name == "supervisor":
                print("\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
                        print(f"Tool calls planned: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print("\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n{'='*60}")
                    print("FINAL REPORT GENERATED")
                    print(f"{'='*60}\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Custom wellness research workflow completed!")
    print("="*60)

# Run the research
await run_research()


Starting custom wellness research workflow...


Node: clarify_with_user

Clarification step output:
I have sufficient information to create your evidence-based 8-week aerobic base improvement plan. I understand you're a beginner runner who can currently run ~30 minutes continuously at ~6:45 min/km pace, runs 3 times per week, and does home workouts. Your goal is gradual improvement without injury. I will now research and develop a comprehensive plan that includes weekly running structure, intensity guidance, strength training recommendations, injury prevention tips, and safety protocols with proper references.

Node: write_research_brief

Research brief generated:
I need an evidence-based 8-week aerobic base improvement plan for a beginner runner who can currently run approximately 30 minutes continuously at a pace of 6:45 minutes per kilometer. I currently run 3 times per week and do home workouts using bodyweight exercises and dumbbells. My primary goal is gradual improvement without

# Evidence-Based 8-Week Aerobic Base Improvement Plan for Beginner Runners

## Current Fitness Assessment and Training Foundation

Based on your current ability to run 30 minutes continuously at 6:45 min/km pace with 3 weekly running sessions, you demonstrate a solid foundation for aerobic base development. The American College of Sports Medicine recommends that beginner runners focus primarily on time-based progression rather than intensity-based training during the initial months of running [1]. Your current pace suggests you're running at approximately 65-75% of maximum heart rate, which aligns well with aerobic base development zones.

Research from exercise physiologist Jack Daniels indicates that aerobic base building should comprise 80-85% of total training volume for developing runners, with the remainder allocated to higher intensity work [2]. This principle will guide the following 8-week progression plan.

## Week-by-Week Running Structure

### Weeks 1-2: Foundation Stabilization
- **Frequency**: 3 runs per week (maintain current schedule)
- **Duration**: 30, 25, 35 minutes respectively
- **Weekly Volume**: 90 minutes total
- **Rest Days**: Minimum 1 full rest day between runs

### Weeks 3-4: Gradual Volume Increase
- **Frequency**: 3-4 runs per week (add optional 4th easy run)
- **Duration**: 35, 30, 40, 20 minutes (if 4 runs) or 35, 30, 40 minutes (if 3 runs)
- **Weekly Volume**: 105-125 minutes total
- **Progression**: 10-15% weekly increase following the established 10% rule [3]

### Weeks 5-6: Volume Consolidation with Tempo Introduction
- **Frequency**: 4 runs per week
- **Duration**: 40, 25, 45, 25 minutes
- **Weekly Volume**: 135 minutes total
- **Structure**: Include one tempo segment (10 minutes at moderate effort) within longest run

### Weeks 7-8: Peak Base Building
- **Frequency**: 4 runs per week
- **Duration**: 45, 30, 50, 25 minutes
- **Weekly Volume**: 150 minutes total
- **Structure**: Two runs with tempo segments, maintaining aerobic emphasis

The International Association of Athletics Federations emphasizes that volume increases should be gradual and sustainable, with recovery weeks incorporated every 3-4 weeks for optimal adaptation [4].

## Detailed Intensity Guidance

### Easy Effort (80% of total running time)
- **Pace**: 7:15-7:45 min/km (45-60 seconds slower than current pace)
- **Heart Rate**: 65-75% maximum HR (approximately 130-150 bpm for most beginners)
- **Perceived Exertion**: 3-4 out of 10, conversational pace
- **Purpose**: Aerobic base development, mitochondrial adaptation, capillary density increase

### Moderate Effort (15% of total running time)
- **Pace**: 6:30-6:45 min/km (current pace to 15 seconds faster)
- **Heart Rate**: 75-85% maximum HR (approximately 150-170 bpm)
- **Perceived Exertion**: 5-6 out of 10, slightly labored breathing
- **Purpose**: Lactate threshold development, aerobic power improvement

### Harder Effort (5% of total running time)
- **Pace**: 6:00-6:20 min/km (25-45 seconds faster than current pace)
- **Heart Rate**: 85-90% maximum HR (approximately 170-180 bpm)
- **Perceived Exertion**: 7-8 out of 10, difficult to maintain conversation
- **Purpose**: VO2max improvement, neuromuscular adaptation

Research from the Norwegian University of Science and Technology demonstrates that polarized training distribution (80% easy, 20% moderate to hard) produces superior aerobic adaptations compared to threshold-heavy approaches in developing endurance athletes [5].

## Strength Training Recommendations

### Upper Body (2x per week)
- **Push-ups**: 2-3 sets of 8-15 reps (progress to decline variations)
- **Dumbbell rows**: 2-3 sets of 10-12 reps each arm
- **Overhead press**: 2 sets of 8-12 reps with dumbbells
- **Plank variations**: 2 sets of 30-60 seconds

### Lower Body and Core (2x per week)
- **Single-leg squats**: 2-3 sets of 6-10 reps each leg
- **Lunges with dumbbells**: 2 sets of 10-12 reps each leg
- **Calf raises**: 2 sets of 15-20 reps (single leg progression)
- **Glute bridges**: 2 sets of 12-15 reps
- **Side planks**: 2 sets of 20-40 seconds each side
- **Dead bugs**: 2 sets of 10 reps each side

The American College of Sports Medicine recommends 2-3 strength training sessions per week for runners, emphasizing functional movement patterns that support running mechanics [6]. Schedule strength training on easy running days or rest days, avoiding placement before long runs or tempo sessions.

## Injury Prevention Strategies

### Pre-Run Warm-Up Protocol (8-10 minutes)
- **Dynamic leg swings**: Forward/back and side-to-side (10 each direction)
- **Walking lunges**: 10 steps each leg
- **High knees**: 20 steps
- **Butt kicks**: 20 steps
- **Ankle circles**: 10 each direction, each foot
- **Gradual running acceleration**: Start at walking pace, gradually increase to easy running pace over 5 minutes

### Post-Run Cool-Down Protocol (10-12 minutes)
- **Walking**: 5 minutes at decreasing pace
- **Static stretching**: Hold each stretch for 30 seconds
  - Calf stretch (gastrocnemius and soleus)
  - Quadriceps stretch
  - Hamstring stretch
  - Hip flexor stretch
  - IT band stretch
- **Foam rolling** (if available): 1-2 minutes each major muscle group

### Targeted Injury Prevention

#### Calf and Achilles Tendon Protection
- **Eccentric calf raises**: 2 sets of 15 reps, 3-second descent
- **Calf stretching**: 2-3 times daily, 30-second holds
- **Gradual pace progression**: Avoid sudden speed increases
- **Proper footwear**: Replace running shoes every 400-500 miles

#### Shin Splint Prevention
- **Toe walks**: 30 seconds, 2 sets
- **Heel walks**: 30 seconds, 2 sets
- **Alphabet draws with toes**: 2 sets each foot
- **Progressive loading**: Increase weekly mileage by maximum 10%
- **Surface variation**: Mix soft surfaces (trails, tracks) with pavement

#### Knee Injury Prevention
- **Hip strengthening**: Focus on glute medius and maximus
- **Proper running form**: Slight forward lean, midfoot strike, cadence 170-180 steps/minute
- **Recovery monitoring**: Address persistent knee discomfort immediately
- **Cross-training**: Swimming or cycling on easy days when needed

The British Journal of Sports Medicine published comprehensive evidence showing that strength training reduces running injury risk by approximately 50% when implemented consistently [7].

## Recovery and Monitoring Protocols

### Daily Recovery Practices
- **Sleep**: 7-9 hours nightly for optimal adaptation
- **Hydration**: Monitor urine color (pale yellow indicates adequate hydration)
- **Nutrition**: Consume carbohydrates within 30 minutes post-run, maintain adequate protein intake (1.2-1.6g/kg body weight daily)
- **Heart rate variability**: Track resting heart rate; increases >10 bpm may indicate inadequate recovery

### Weekly Recovery Assessment
- **Subjective wellness questionnaire**: Rate sleep quality, muscle soreness, motivation, and stress levels (1-5 scale)
- **Running pace consistency**: Easy runs should feel progressively easier at same pace
- **Strength maintenance**: Ability to complete prescribed strength exercises without significant fatigue

## Safety Guidelines and Training Modification

### Immediate Stop Indicators
- **Sharp, localized pain** during or after running
- **Chest pain, dizziness, or unusual breathlessness** during exercise
- **Swelling, heat, or persistent tenderness** in joints or muscles
- **Gait alterations** due to pain or discomfort
- **Illness symptoms**: Fever, persistent cough, or systemic fatigue

### Program Modification Protocols

#### Minor Discomfort Management
- **Reduce training volume by 20-30%** while maintaining frequency
- **Eliminate tempo and harder efforts**, focus on easy-paced running
- **Increase recovery time between sessions**
- **Implement additional stretching and strengthening for affected areas**

#### Persistent Issues (>3 days)
- **Take 2-3 complete rest days** from running
- **Maintain cross-training** with low-impact activities (swimming, cycling)
- **Continue strength training** if not contraindicated
- **Consult healthcare provider** if symptoms persist beyond one week

#### Return to Training Protocol
- **Resume at 50% previous volume** after symptom resolution
- **Progress weekly increases to 5-8%** rather than 10%
- **Monitor symptoms closely** for recurrence
- **Prioritize sleep, nutrition, and stress management**

The International Association of Athletics Federations emphasizes that training modifications should be implemented promptly to prevent minor issues from becoming major injuries [8].

### Progressive Overload Principles

Training adaptations occur through systematic application of progressive overload while respecting individual recovery capacity. Research from the Scandinavian Journal of Medicine and Science in Sports demonstrates that aerobic base improvements continue for 12-16 weeks in beginning runners when training load is managed appropriately [9].

Monitor weekly training stress through:
- **Total running time progression**
- **Subjective exertion ratings**
- **Heart rate response consistency**
- **Recovery quality markers**

This 8-week plan provides a evidence-based foundation for aerobic development while minimizing injury risk through systematic progression, appropriate intensity distribution, and comprehensive recovery monitoring.

### Sources

[1] American College of Sports Medicine Position Stand: Physical Activity and Public Health Guidelines: https://www.acsm.org/docs/default-source/files-for-resource-library/guidelines-download.pdf

[2] Daniels, Jack. Daniels' Running Formula, 3rd Edition. Human Kinetics, 2014.

[3] Nielsen RO, et al. Training errors and running related injuries: a systematic review. International Journal of Sports Physical Therapy. 2012;7(1):58-75.

[4] International Association of Athletics Federations Training Guidelines: https://www.worldathletics.org/development/education

[5] Seiler S. What is best practice for training intensity and duration distribution in endurance athletes? International Journal of Sports Physiology and Performance. 2010;5(3):276-291.

[6] American College of Sports Medicine Position Stand: Progression Models in Resistance Training for Healthy Adults. Medicine & Science in Sports & Exercise. 2009;41(3):687-708.

[7] Lauersen JB, Bertelsen DM, Andersen LB. The effectiveness of exercise interventions to prevent sports injuries: a systematic review and meta-analysis. British Journal of Sports Medicine. 2014;48(11):871-877.

[8] International Association of Athletics Federations Injury Prevention Guidelines: https://www.worldathletics.org/about-iaaf/documents/health-science

[9] Laursen PB, Jenkins DG. The scientific basis for high-intensity interval training: optimising training programmes and maximising performance in highly trained endurance athletes. Sports Medicine. 2002;32(1):53-73.


Custom wellness research workflow completed!


#### Step 2 - second run – updated question: Custom Wellness Research Request

In [30]:
from IPython.display import Markdown, display

# Create our custom wellness research request
research_request = """
Create an evidence-based 8-week aerobic base plan for a 42-year-old recreational runner, 
currently running ~30 minutes at ~6:45 min/km, with knee pain—prioritizing joint-friendly progression, recovery, and injury risk reduction.

Context:
- Runs 3 times per week
- Does home workouts (bodyweight / dumbbells)
- Goal is gradual improvement without injury

Please include:
- Week-by-week running structure
- Intensity guidance (easy vs moderate vs harder efforts)
- Simple strength training recommendations
- Injury prevention tips (calves, shins, knees)
- Safety notes and when to stop training
- Sources / references
"""

# Execute the graph
async def run_research():
    """Run the custom wellness research workflow and display results."""
    print("Starting custom wellness research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        custom_config,
        stream_mode="updates"
    ):
        # Display progress for each node
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\nClarification step output:")
                    print(last_msg.content)
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print("\nResearch brief generated:")
                    print(node_output["research_brief"][:500] + "...")
            
            elif node_name == "supervisor":
                print("\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
                        print(f"Tool calls planned: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print("\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n{'='*60}")
                    print("FINAL REPORT GENERATED")
                    print(f"{'='*60}\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Custom wellness research workflow completed!")
    print("="*60)

# Run the research
await run_research()

Starting custom wellness research workflow...


Node: clarify_with_user

Clarification step output:
I have sufficient information to create your evidence-based 8-week aerobic base plan. I understand you're a 42-year-old recreational runner currently doing 30-minute runs at ~6:45 min/km pace, 3 times per week, with existing knee pain. You want a joint-friendly progression plan that prioritizes injury prevention and gradual improvement, including weekly running structure, intensity guidance, strength training recommendations, and injury prevention strategies. I will now begin researching current evidence-based practices for aerobic base building, injury prevention protocols, and safe training progressions for runners with knee issues to create your comprehensive 8-week plan.

Node: write_research_brief

Research brief generated:
I need a comprehensive, evidence-based 8-week aerobic base building plan for a 42-year-old recreational runner currently running approximately 30 minutes at 6:45

# Evidence-Based 8-Week Aerobic Base Building Plan for Recreational Runners with Knee Pain

## Overview and Training Philosophy

Aerobic base building for recreational runners over 40 with existing knee pain requires a careful balance between progressive overload and injury prevention. Research demonstrates that 80-90% of training should occur at low intensities (Zone 1-2) to maximize aerobic adaptations while minimizing injury risk [1]. The polarized training model, emphasizing high-volume easy running with minimal moderate-intensity work, has been shown to be particularly effective for recreational runners and those managing joint issues [2].

For runners with knee pain, the primary focus must be on gradual volume increases of no more than 10% per week, proper load management, and integration of strength training to address biomechanical deficiencies [3]. Studies show that runners over 40 benefit from increased recovery time between sessions and lower training frequencies compared to younger athletes [4].

## Week-by-Week Running Structure

### Weeks 1-2: Foundation Phase
**Frequency:** 3 runs per week
**Weekly Volume:** 
- Week 1: 90 minutes total (3 x 30 minutes)
- Week 2: 95 minutes total (increase by 5 minutes)

**Session Structure:**
- Run 1: Easy pace, maintain current 30-minute duration
- Run 2: Easy pace, 30 minutes + 2-3 minutes additional in Week 2
- Run 3: Easy pace, 30 minutes

**Rest Days:** 48 hours between runs minimum

### Weeks 3-4: Early Development
**Frequency:** 3-4 runs per week (add 4th run in Week 4 if knee pain remains stable)
**Weekly Volume:**
- Week 3: 105 minutes total
- Week 4: 115 minutes total

**Session Structure:**
- Run 1: Easy continuous run (35 minutes)
- Run 2: Easy pace with 4 x 30-second pickups (35 minutes total)
- Run 3: Easy recovery run (30 minutes)
- Run 4 (Week 4 only): Easy short run (20 minutes)

### Weeks 5-6: Base Building
**Frequency:** 4 runs per week
**Weekly Volume:**
- Week 5: 125 minutes total
- Week 6: 135 minutes total

**Session Structure:**
- Run 1: Easy long run (45-50 minutes)
- Run 2: Easy pace with 6 x 20-second strides (35 minutes total)
- Run 3: Easy recovery run (25 minutes)
- Run 4: Easy-moderate tempo run (25-30 minutes with 10 minutes at moderate effort)

### Weeks 7-8: Consolidation Phase
**Frequency:** 4 runs per week
**Weekly Volume:**
- Week 7: 145 minutes total
- Week 8: 140 minutes total (slight reduction for recovery)

**Session Structure:**
- Run 1: Easy long run (55 minutes)
- Run 2: Fartlek session (35 minutes with 8 x 1-minute moderate efforts)
- Run 3: Easy recovery run (25-30 minutes)
- Run 4: Easy-moderate continuous run (30 minutes)

## Intensity Guidance and Heart Rate Zones

### Heart Rate Zone Calculation
For a 42-year-old runner, maximum heart rate is approximately 178 bpm (220 - age). Training zones are:

**Zone 1 (Easy/Recovery): 60-70% HRmax (107-125 bpm)**
- Perceived Exertion: 3-4/10
- Can hold full conversation
- Nasal breathing possible
- Primary training zone (80-85% of total volume)

**Zone 2 (Aerobic Base): 70-80% HRmax (125-142 bpm)**
- Perceived Exertion: 5-6/10
- Can speak in short sentences
- Comfortable but purposeful effort
- Secondary training zone (10-15% of total volume)

**Zone 3 (Moderate/Tempo): 80-85% HRmax (142-151 bpm)**
- Perceived Exertion: 7/10
- Can speak only a few words
- Comfortably hard effort
- Limited use (5% of total volume maximum)

### Pace Guidelines
Research indicates that easy pace should be 60-90 seconds per kilometer slower than current race pace [5]. Given the current 6:45 min/km pace:
- Easy runs: 7:45-8:45 min/km
- Moderate efforts: 6:15-6:45 min/km
- Strides/pickups: Natural acceleration, not all-out

## Strength Training Program

### Frequency and Timing
Strength training should be performed 2-3 times per week on non-consecutive days, ideally after easy runs or on rest days [6]. Research shows that runners benefit most from strength training focused on posterior chain, core stability, and single-leg strength [7].

### Phase 1 Routine (Weeks 1-4)
**Frequency:** 2x per week
**Duration:** 20-25 minutes

**Circuit A:**
- Bodyweight squats: 2 sets x 12-15 reps
- Single-leg glute bridges: 2 sets x 8-10 per leg
- Calf raises: 2 sets x 15-20 reps
- Plank: 2 sets x 30-45 seconds
- Clamshells: 2 sets x 10-12 per side

**Circuit B:**
- Wall sit: 2 sets x 20-30 seconds
- Single-leg calf raises: 2 sets x 8-10 per leg
- Side plank: 2 sets x 20-30 seconds per side
- Bird dog: 2 sets x 8-10 per side
- Monster walks: 2 sets x 10 steps each direction

### Phase 2 Routine (Weeks 5-8)
**Frequency:** 2-3x per week
**Duration:** 25-30 minutes

**Circuit A:**
- Dumbbell goblet squats: 3 sets x 10-12 reps
- Single-leg Romanian deadlifts: 3 sets x 6-8 per leg
- Single-leg calf raises: 3 sets x 10-12 per leg
- Plank with leg lifts: 3 sets x 6-8 per leg
- Lateral lunges: 3 sets x 8-10 per side

**Circuit B:**
- Step-ups (using sturdy chair): 3 sets x 8-10 per leg
- Dumbbell single-arm row: 3 sets x 10-12 per arm
- Single-leg glute bridges: 3 sets x 10-12 per leg
- Russian twists: 3 sets x 20 total
- Hip hikes: 3 sets x 8-10 per side

## Injury Prevention Strategies

### Knee-Specific Prevention
Research identifies several key factors in knee injury prevention for runners [8]:

**Biomechanical Focus:**
- Maintain cadence between 170-180 steps per minute to reduce impact forces
- Focus on midfoot landing rather than heel striking
- Avoid overstriding by increasing turnover rate

**Strength Training Priorities:**
- Hip abductor strengthening reduces knee valgus stress
- Quadriceps and glutes strengthening improves patellofemoral tracking
- Single-leg stability work addresses functional deficits

### Calf and Shin Protection
**Calf Injury Prevention:**
- Gradual volume increases prevent overuse injuries
- Eccentric calf strengthening (negative calf raises) 2-3x per week
- Dynamic warm-up including calf raises and ankle circles

**Shin Splint Prevention:**
- Avoid sudden increases in training intensity or surface changes
- Strengthen tibialis anterior with toe walks and resistance band exercises
- Ensure proper footwear with adequate cushioning and support

### Recovery Protocols
**Daily Recovery Practices:**
- 10-15 minutes post-run stretching focusing on calves, hamstrings, and hip flexors
- Ice application to knees for 10-15 minutes after runs if pain is present
- Elevation of legs for 10-15 minutes post-exercise

**Weekly Recovery Sessions:**
- 20-30 minute gentle yoga or stretching session
- Self-massage using foam roller or massage ball
- Cross-training alternatives: swimming, cycling, or elliptical if running causes pain

## Safety Protocols and Warning Signs

### Immediate Stop Indicators
Research establishes clear criteria for stopping training to prevent serious injury [9]:

**Pain-Related Warning Signs:**
- Sharp, acute pain during or after running
- Pain that worsens during a run
- Pain that interferes with normal walking
- Pain that persists for more than 2-3 days after running
- Swelling or visible inflammation around joints

**Systemic Warning Signs:**
- Persistent fatigue lasting more than 48 hours
- Elevated resting heart rate (10+ bpm above normal)
- Sleep disturbances or mood changes
- Decreased performance despite adequate recovery

### Pain Management Guidelines
**Acceptable Training Pain Levels:**
- Mild discomfort (1-3/10 pain scale) that doesn't worsen during activity
- Stiffness that improves with gentle movement
- Muscle soreness that resolves within 24-48 hours

**Modification Protocols:**
- If knee pain reaches 4/10 or higher, reduce training volume by 20-30%
- If pain persists for 3 consecutive training days, take 2-3 complete rest days
- Consider cross-training alternatives if running pain exceeds 3/10

### Progressive Return Guidelines
After any training interruption due to pain:
- Resume at 70% of previous volume
- Increase by no more than 10% per week
- Monitor pain levels daily using 1-10 scale
- Seek professional evaluation if pain persists beyond 7-10 days

## Training Load Management

### Monitoring and Adjustment
**Weekly Assessment Protocol:**
- Rate overall fatigue (1-10 scale) each Sunday
- Track knee pain levels before, during, and after each run
- Monitor sleep quality and duration
- Record resting heart rate trends

**Load Adjustment Criteria:**
- If weekly fatigue rating exceeds 7/10, reduce next week's volume by 15-20%
- If knee pain trends upward over 3 consecutive sessions, implement rest day
- If performance declines despite adequate recovery, consider deload week

**Environmental and Surface Considerations:**
- Prioritize softer surfaces (trails, tracks, treadmill) over concrete
- Adjust training intensity based on weather conditions
- Consider indoor alternatives during extreme temperatures

The 8-week progression emphasizes gradual adaptation while respecting the constraints of existing knee pain. Success depends on consistent monitoring, appropriate load management, and willingness to modify the plan based on individual response. The integration of strength training and injury prevention strategies provides a comprehensive approach to long-term running health and performance improvement.

### Sources

[1] Seiler, S. (2010). What is best practice for training intensity and duration distribution in endurance athletes? International Journal of Sports Physiology and Performance, 5(3), 276-291.

[2] Laursen, P. B., & Buchheit, M. (2019). Science and application of high-intensity interval training. Human Kinetics.

[3] Nielsen, R. O., et al. (2012). Training errors and running related injuries: A systematic review. International Journal of Sports Physical Therapy, 7(1), 58-75.

[4] Tanaka, H., & Seals, D. R. (2008). Endurance exercise performance in Masters athletes: Age-associated changes and underlying physiological mechanisms. Journal of Physiology, 586(1), 55-63.

[5] Daniels, J. (2014). Daniels' Running Formula. Human Kinetics.

[6] Beattie, K., et al. (2014). The effect of strength training on performance in endurance athletes. Sports Medicine, 44(6), 845-865.

[7] Lauersen, J. B., et al. (2014). The effectiveness of exercise interventions to prevent sports injuries: A systematic review and meta-analysis. British Journal of Sports Medicine, 48(11), 871-877.

[8] Van Gent, R. N., et al. (2007). Incidence and determinants of lower extremity running injuries in long distance runners: A systematic review. British Journal of Sports Medicine, 41(8), 469-480.

[9] Hreljac, A. (2004). Impact and overuse injuries in runners. Medicine & Science in Sports & Exercise, 36(5), 845-849.


Custom wellness research workflow completed!


## 💡Output Review (What Worked Well vs. Improvements)

**What worked well**
- The system produced a structured, actionable 8-week plan (week-by-week schedule, intensity guidance, strength, injury prevention, and safety rules).
- Even with conservative limits (1 iteration, 2 tool calls), the report remained coherent and comprehensive.
- Including a sources section improves trust and makes key claims easier to validate.

**What could be improved**
- Personalization could be stronger with a short clarification step (age, injury history, footwear, typical weekly schedule, preferred training days).
- Source governance: citations could prioritize sports medicine guidelines and beginner-focused endurance training sources; a few references (e.g., HIIT-focused papers) may be less directly relevant to aerobic base building for beginners.
- Add a simple tracking/check-in template (weekly minutes, RPE, soreness score, sleep quality) to help the user iterate and adjust load safely.

